In [1]:
%matplotlib inline

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.preprocessing as sp

In [3]:
Dataset = np.load('Dataset.npy')

In [4]:
model = tf.keras.models.load_model('Final_model.h5')

In [5]:
Data_False = np.load('Data_False.npy')
X_False = Data_False[:9]
Y_False = Data_False[-3:]

X_train       = np.load('X_train.npy')
X_train_scl   = np.load('X_train_scl.npy')
X_train_stdev = np.load('X_train_stdev.npy')
X_train_mean  = np.load('X_train_mean.npy')
Y_train_stdev = np.load('Y_train_stdev.npy')
Y_train_mean  = np.load('Y_train_mean.npy')

In [6]:
FR_min = float(np.min(X_train[:,1]))
FR_max = float(np.max(X_train[:,1]))
RR_min = float(np.min(X_train[:,5]))
RR_max = float(np.max(X_train[:,5]))
B_min  = float(np.min(X_train[:,6]))
B_max  = float(np.max(X_train[:,6]))
QR_min = float(np.min(X_train[:,7]))
QR_max = float(np.max(X_train[:,7]))

In [7]:
def Standardization(Data):
    scaler = sp.StandardScaler().fit(Data)
    scl = scaler.transform(Data)
    return scl , scaler

def Predict(X_try):
    X_try_scl = (X_try - X_train_mean) / X_train_stdev
    Y_predict_scl = model.predict(X_try_scl)
    Y_predict = Y_predict_scl * Y_train_stdev + Y_train_mean
    XME_D = Y_predict[0,1]
    return XME_D

def Err(X_try,X_False):
    Err1 = ((X_try[0,1] - X_False[0,1]) ** 2) * (1/0.480704)
    Err2 = ((X_try[0,5] - X_False[0,5]) ** 2) * (1/0.584310)
    Err3 = ((X_try[0,6] - X_False[0,6]) ** 2) * (1/0.503303)
    Err4 = ((X_try[0,7] - X_False[0,7]) ** 2) * (1/0.480690)
    Error = Err1 + Err2 + Err3 + Err4
    return Error

def Step(i,iteration):
    if i <= iteration * 0.1:
        j = 1
    elif i > iteration * 0.1 and i <= iteration * 0.5:
        j = 0.8
    elif i > iteration * 0.5:
        j = 0.5
    return j

def check(item,LB,RB):
    value = float(X_try[0,item])
    if value < LB:
        value = LB
    elif value > RB:
        value = RB
    else:
        value = value
    return value

In [8]:
X_False = np.reshape(X_False , (1,9))
X_try = X_False.copy()

Error_0 = 100000000000000
print(X_False)
print(Y_False[1])

XME_D_new = 1
XME_D_old = 0
iteration = 1000

for i in range(iteration):
    j = Step(i,iteration)
    X_try[0,1] = X_False[0,1] + 0.044259 * j * random.uniform(-1,1) #FR
    X_try[0,5] = X_False[0,5] + 0.019979 * j * random.uniform(-1,1) #RR
    X_try[0,6] = X_False[0,6] + 0.994289 * j * random.uniform(-1,1) #B
    X_try[0,7] = X_False[0,7] + 29.15039 * j * random.uniform(-1,1) #QR
    #if X_try <最小值的話 要等於最小值
    X_try[0,1] = check(1,FR_min,FR_max)
    X_try[0,5] = check(5,RR_min,RR_max)
    X_try[0,6] = check(6,B_min,B_max)
    X_try[0,7] = check(7,QR_min,QR_max)
    XME_D_new = Predict(X_try)
    #print('-' , end = '')
    print(str(i) + '--------------------')
    print(X_False)
    print(X_try)
    print(XME_D_new)
    if XME_D_new >= 0.9900:
        Error = Err(X_try,X_False)
        X_False = X_try.copy()
        if Error < Error_0:
            X_ans = X_try.copy()
            Error_0 = Error
            XME_D_ans = XME_D_new
    else:
        if XME_D_new >= XME_D_old:
            X_False = X_try.copy()
            XME_D_old = XME_D_new
print('ANS:')
print(X_ans)
print(XME_D_ans)

[[ 2.01600000e+02  2.63000000e+00  3.80000000e+01  2.33000000e+01
  -4.16123817e+03  1.23800000e+00  9.73300000e+01  2.72111058e+03
  -2.52787842e+03]]
0.979826368
0--------------------
[[ 2.01600000e+02  2.63000000e+00  3.80000000e+01  2.33000000e+01
  -4.16123817e+03  1.23800000e+00  9.73300000e+01  2.72111058e+03
  -2.52787842e+03]]
[[ 2.01600000e+02  2.63478406e+00  3.80000000e+01  2.33000000e+01
  -4.16123817e+03  1.25528533e+00  9.65581480e+01  2.71269064e+03
  -2.52787842e+03]]
0.9744454787342289
1--------------------
[[ 2.01600000e+02  2.63478406e+00  3.80000000e+01  2.33000000e+01
  -4.16123817e+03  1.25528533e+00  9.65581480e+01  2.71269064e+03
  -2.52787842e+03]]
[[ 2.01600000e+02  2.65183943e+00  3.80000000e+01  2.33000000e+01
  -4.16123817e+03  1.25947905e+00  9.64707297e+01  2.69110323e+03
  -2.52787842e+03]]
0.974448067847853
2--------------------
[[ 2.01600000e+02  2.65183943e+00  3.80000000e+01  2.33000000e+01
  -4.16123817e+03  1.25947905e+00  9.64707297e+01  2.691103

In [9]:
print('        ' + 'Current' + '      ' + 'Target')
print('Tin' + '     ' + str(Data_False[0]) + '        ' + str(X_ans[0,0]))
print('FR' + '      ' + str(Data_False[1]) + '         ' + str(round(X_ans[0,1],2)))
print('T1' + '      ' + str(Data_False[2]) + '         ' + str(X_ans[0,2]))
print('T2' + '      ' + str(Data_False[3]) + '         ' + str(X_ans[0,3]))
print('Qcool' + '   ' + str(round(Data_False[4],2)) + '     ' + str(round(X_ans[0,4],2)))
print('RR' + '      ' + str(Data_False[5]) + '        ' + str(round(X_ans[0,5],3)))
print('B' + '       ' + str(Data_False[6]) + '        ' + str(round(X_ans[0,6],2)))
print('QR' + '      ' + str(round(Data_False[7],2)) + '      ' + str(round(X_ans[0,7],2)))
print('Qc' + '      ' + str(round(Data_False[8],2)) + '     ' + str(round(X_ans[0,8],2)))
print('----------------------------------')
print('XME_D' + '   ' + str(round(Data_False[-2],4)) + '       ' + str(round(XME_D_ans,4)))

        Current      Target
Tin     201.6        201.6
FR      2.63         2.72
T1      38.0         38.0
T2      23.3         23.3
Qcool   -4161.24     -4161.24
RR      1.238        1.437
B       97.33        102.26
QR      2721.11      2698.75
Qc      -2527.88     -2527.88
----------------------------------
XME_D   0.9798       0.9907
